In [1]:
include("./src/MPO_common.jl")
using .MPO_common
using JLD2
using ITensors
using QuadGK
using Statistics
using LinearAlgebra
include("./src/iDMRG.jl")
using .iDMRG

We have chosen the same convention as Fig.3 in Kitaev's paper. The periodic direction (which we call $L_y$) is along $n_1=(-\frac{1}{2},-\frac{\sqrt{3}}{2})$.

In [2]:
Ly = 3 # Here it denotes number of true unit-cells along the Ly direction.
uc = 2 * Ly
Jx = -0.4
Jy = -0.4
Jz = -1.0

-1.0

Below is the analytic energy

In [3]:
dispersion(a1, a2) = abs(Jx * exp(im * 2pi * a1) + Jy * exp(im * 2pi * a2) + Jz)

function integrate_dispersion(Ly)
    alphas = (1:Ly) ./ Ly
    quadgk(0.0, 1.0) do a2
        mean([dispersion(a1, a2) for a1 in alphas]) / 2
    end
end

# usage
val, err = integrate_dispersion(Ly)

(0.5442204976647842, 1.644663294442239e-10)

In [4]:
oplist = MPOsum()
for i = 1:Ly
    mpoadd!(oplist, 4 * Jx, (2 * i - 1, "Sx"), (2 * i, "Sx"))
    mpoadd!(oplist, 4 * Jy, (2 * i - 1, "Sy"), (2 * i + 2 * Ly, "Sy"))
end

for i = 1:Ly
    mpoadd!(oplist, 4 * Jz, (2 * i, "Sz"), (mod1(2 * i + 1, 2 * Ly), "Sz"))
end

sites = siteinds("S=1/2", uc)
iH = op_to_hm_inf(oplist, sites);

Note that we have chosen a larger krylovdimmax and niter. These are the crucial parameters to tune for a convergence. In general I think for gapped phases they can be tuned a bit higher so that the convergence is faster (although each sweep would take longer time). For gapless phase they can be lower to achieve convergence, since we don't want to use bad trial states to contract environment too much.

If the $noiseonoff$ is on, then at each global sweep, a sweep with noise (following Hubig et al. strictly single-site DMRG paper) turned on is first performed, then sweeps without noise are performed.

In [5]:
finalstate = idmrg_run(iH, sites; maxdimlist=[50, 100], sweeplist=[10, 20], noise=1e-2,krylovdimmax=5,niter=3,noiseonoff=true, cutoff=1e-10);

finish initialization
Maximum bond dimension is: 1
Energy is: -0.5425597322249013
<Λ(n)|Λ(n+1)> overlap is: 1.0000000000000002
== Global sweep 1 ==
Maximum bond dimension is: 4
Energy is: -0.6775261104492998
<Λ(n)|Λ(n+1)> overlap is: 0.7044210947274444
  Sweep 1
Maximum bond dimension is: 6
Energy is: -0.530425642459028
<Λ(n)|Λ(n+1)> overlap is: 0.9998645576640885
  Sweep 2
Maximum bond dimension is: 16
Energy is: -0.5372748893195377
<Λ(n)|Λ(n+1)> overlap is: 0.9962219124165398
  Sweep 3
Maximum bond dimension is: 28
Energy is: -0.5437074322439219
<Λ(n)|Λ(n+1)> overlap is: 0.9977860641294674
  Sweep 4
Maximum bond dimension is: 50
Energy is: -0.543736067369687
<Λ(n)|Λ(n+1)> overlap is: 0.9995911773308594
  Sweep 5
Maximum bond dimension is: 50
Energy is: -0.5437918248899921
<Λ(n)|Λ(n+1)> overlap is: 0.9925267302623647
  Sweep 6
Maximum bond dimension is: 50
Energy is: -0.5438616535593918
<Λ(n)|Λ(n+1)> overlap is: 0.9922556477665248
  Sweep 7
Maximum bond dimension is: 50
Energy is: -0.

The convergence criteria $<\Lambda(n)|\Lambda(n+1)>$ follows IP McCulloch arXiv:0804.2509.

The GS energy computed from iDMRG is $-0.544220420851436$, which is quite accurate.

In [9]:
(0.5442204976647842-0.544220420851436)/0.544220420851436

1.4114381837682496e-7

In [10]:
ψA,L1=left_canonical_form(finalstate,sites); # I used an iteration algorithm so the output is a convergence criteria which is not material.

Converged after 2 sweeps with error 9.908518949546789e-9
Below is the result
0.999999317391789 + 1.002966134064314e-9im


In [11]:
ψB,Rl=right_canonical_form(finalstate,sites); 

Converged after 12 sweeps with error 6.468709686643227e-9
Below is the result
0.999999998946195 - 1.511298631739009e-11im
